In [17]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation

In [5]:
def UTK_read(path_folder):
    image_list = []
    gender_list = []
    age_list = []

    for file in os.listdir(path_folder):
        
        parts = file.split('_')
        gender = int(parts[1])
        age = int(parts[0])
        
        path_image = os.path.join(path_folder, file)

        image = cv2.imread(path_image)
        image = cv2.resize(image,(64,64))
        image = image / 255.0

        image_list.append(image)
        gender_list.append(gender)
        age_list.append(age)

    X = np.array(image_list)
    y_gender = np.array(gender_list)
    y_age = np.array(age_list)
    return X, y_gender, y_age

In [6]:
utk_folder = '/kaggle/input/gender-age-recognition/UTKFace'
X, y_gender, y_age = UTK_read(utk_folder)

In [7]:
X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test = train_test_split(X, y_age, y_gender, test_size = 0.3, random_state = 42)

datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

In [28]:
input_shape = (64, 64, 3)
inputs = Input(shape=input_shape)


# Block 1
x = Conv2D(32, (3, 3), padding='same')(inputs)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)


# Block 2
x = Conv2D(64, (3, 3), padding='same')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)

# Block 3
x = Conv2D(128, (3, 3), padding='same')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)


# Block 4
x = Conv2D(256, (3, 3), padding='same')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)

x = Conv2D(512, (3, 3), padding='same')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)


x = Flatten()(x)
x = Dropout(0.1)(x)

output_gender = Dense(1, activation='sigmoid', name='gender')(x)
output_age = Dense(1, activation='linear', name='age')(x)

model = Model(inputs=inputs, outputs=[output_gender, output_age])
model.compile(optimizer='adam',
              loss={'gender': 'binary_crossentropy', 'age': 'mse'},
              metrics={'gender': 'accuracy', 'age': 'mae'})

model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10            │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_33 (Conv2D)        │ (None, 64, 64, 32)     │            896 │ input_layer_10[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_26             │ (None, 64, 64, 32)     │              0 │ conv2d_33[0][0]        │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_28    │ (None, 64, 64, 32)     │            128 │ activation_26[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_30          │ (None, 32, 32, 32)     │              0 │ batch_normalization_2… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_34 (Conv2D)        │ (None, 32, 32, 64)     │         18,496 │ max_pooling2d_30[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_27             │ (None, 32, 32, 64)     │              0 │ conv2d_34[0][0]        │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_29    │ (None, 32, 32, 64)     │            256 │ activation_27[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_31          │ (None, 16, 16, 64)     │              0 │ batch_normalization_2… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_35 (Conv2D)        │ (None, 16, 16, 128)    │         73,856 │ max_pooling2d_31[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_28             │ (None, 16, 16, 128)    │              0 │ conv2d_35[0][0]        │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_30    │ (None, 16, 16, 128)    │            512 │ activation_28[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_32          │ (None, 8, 8, 128)      │              0 │ batch_normalization_3… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_36 (Conv2D)        │ (None, 8, 8, 256)      │        295,168 │ max_pooling2d_32[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_29        

 Total params: 1,576,642 (6.01 MB)

 Trainable params: 1,574,658 (6.01 MB)

 Non-trainable params: 1,984 (7.75 KB)

In [29]:
history = model.fit(
    X_train,
    {'gender': y_gender_train, 'age': y_age_train},
    validation_data=(X_test, {'gender': y_gender_test, 'age': y_age_test}),
    epochs=40,
    batch_size=32
)

Epoch 1/40
519/519 ━━━━━━━━━━━━━━━━━━━━ 123s 226ms/step - age_loss: 232.2992 - age_mae: 11.2922 - gender_accuracy: 0.6754 - gender_loss: 0.6593 - loss: 232.9583 - val_age_loss: 321.7575 - val_age_mae: 12.9074 - val_gender_accuracy: 0.7600 - val_gender_loss: 0.5034 - val_loss: 321.2062
Epoch 2/40
519/519 ━━━━━━━━━━━━━━━━━━━━ 117s 225ms/step - age_loss: 109.3023 - age_mae: 7.8226 - gender_accuracy: 0.7653 - gender_loss: 0.4935 - loss: 109.7959 - val_age_loss: 135.3369 - val_age_mae: 8.0989 - val_gender_accuracy: 0.7911 - val_gender_loss: 0.4708 - val_loss: 134.9798
Epoch 3/40
519/519 ━━━━━━━━━━━━━━━━━━━━ 117s 226ms/step - age_loss: 78.7402 - age_mae: 6.6404 - gender_accuracy: 0.8078 - gender_loss: 0.4274 - loss: 79.1677 - val_age_loss: 99.7693 - val_age_mae: 7.4320 - val_gender_accuracy: 0.8111 - val_gender_loss: 0.4143 - val_loss: 100.1739
Epoch 4/40
519/519 ━━━━━━━━━━━━━━━━━━━━ 116s 223ms/step - age_loss: 63.5303 - age_mae: 5.9738 - gender_accuracy: 0.8169 - gender_loss: 0.4049 - loss:

In [30]:
model.save('age_gender.keras')